In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras_core as keras
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

Using TensorFlow backend
TensorFlow version: 2.10.1
KerasNLP version: 0.7.0


In [23]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade

# This sample uses Keras Core, the multi-backend version of Keras.
# The selected backend is TensorFlow (other supported backends are 'jax' and 'torch')
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

     -------------------------------------- 101.3/101.3 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 950.8/950.8 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 133.7/133.7 kB 8.2 MB/s eta 0:00:00
   ---------------------------------------- 2.7/2.7 MB 12.3 MB/s eta 0:00:00
   ---------------------------------------- 240.6/240.6 kB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 87.5/87.5 kB 5.2 MB/s eta 0:00:00


Load the disaster tweets and explore the dataset

In [30]:
df_train = pd.read_csv(".//kaggle/train.csv")
df_test = pd.read_csv(".//kaggle/test.csv")

In [22]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


* PREPROCESS THE DATA

* LOAD A DISTILBERT MODEL, FROM KERAS NLP

* TRAIN MODEL, FINE-TUNING BERT

* GENERATE THE SUBMISSION FILE